In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install split-folders tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import numpy as np
import splitfolders
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
base_dir = '/content/drive/MyDrive/Dataset'

Split Dataset into 3 part: Train, Val, and Test

In [5]:
splitfolders.ratio(
    base_dir,
    output='Capstone',
    ratio=(.7, 0.2,0.1)
)

Copying files: 2375 files [01:36, 24.65 files/s]


In [6]:
train_dir = 'Capstone/train'
val_dir = 'Capstone/val' 
test_dir = 'Capstone/test'

os.listdir(train_dir)

['Hispa', 'Healthy', 'BrownSpot', 'LeafBlast']

In [7]:
class_name = ['Healthy', 'Hispa', 'BrownSpot', 'LeafBlast']

Data Augmentation

In [8]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=0.45, 
                    width_shift_range=0.2,
                     height_shift_range=0.2,
                     zoom_range=(0.2), 
                     fill_mode='nearest', 
                    horizontal_flip=True,
                    )

test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [9]:

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150,150), 
    batch_size=45, 
    shuffle=True,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=45,
    shuffle=True,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    batch_size=45,
    shuffle=True,
    class_mode='categorical'
)

Found 1661 images belonging to 4 classes.
Found 474 images belonging to 4 classes.
Found 240 images belonging to 4 classes.


Modeling

In [10]:
from keras.callbacks import EarlyStopping
monitor_val_acc = EarlyStopping(monitor='val_accuracy', patience=3)

In [11]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience = 2,
    verbose=1,
    factor=0.3,
    min_lr=0.000001
)

In [12]:
from tensorflow.keras.applications import InceptionV3
tfmodel = InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(150,150,3))

87910968/87910968 [==============================] - 1s 0us/step


In [13]:
from keras.models import Sequential
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D

In [14]:
model = Sequential()
model.add(tfmodel)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))      
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                        

In [16]:
history = model.fit(train_generator,
                    epochs=50,
                    callbacks=[monitor_val_acc,reduce_lr],
                    validation_data=validation_generator)

Epoch 1/50
37/37 [==============================] - 296s 7s/step - loss: 1.3551 - accuracy: 0.3522 - val_loss: 1.2845 - val_accuracy: 0.4198 - lr: 1.0000e-04
Epoch 2/50
37/37 [==============================] - 271s 7s/step - loss: 1.1903 - accuracy: 0.4738 - val_loss: 1.1074 - val_accuracy: 0.5084 - lr: 1.0000e-04
Epoch 3/50
37/37 [==============================] - 272s 7s/step - loss: 1.0079 - accuracy: 0.5864 - val_loss: 1.2264 - val_accuracy: 0.5021 - lr: 1.0000e-04
Epoch 4/50
37/37 [==============================] - 277s 8s/step - loss: 0.8989 - accuracy: 0.6352 - val_loss: 1.1920 - val_accuracy: 0.5654 - lr: 1.0000e-04
Epoch 5/50
37/37 [==============================] - 278s 8s/step - loss: 0.8477 - accuracy: 0.6604 - val_loss: 0.9352 - val_accuracy: 0.6308 - lr: 1.0000e-04
Epoch 6/50
37/37 [==============================] - 279s 8s/step - loss: 0.7490 - accuracy: 0.7050 - val_loss: 1.0125 - val_accuracy: 0.6329 - lr: 1.0000e-04
Epoch 7/50
37/37 [==============================] - 